In [7]:
import numpy as np
import random

# Define the UAV Route Planning Environment
class Environment:
    def __init__(self, grid_size):
        self.grid_size = grid_size
        self.obstacles = set()

    def add_obstacle(self, position):
        self.obstacles.add(tuple(position))

    def is_obstacle(self, position):
        return tuple(position) in self.obstacles

# Define the UAV Route Representation
class UAVRoute:
    def __init__(self, waypoints):
        self.waypoints = waypoints
        self.fitness = None  # To be calculated

# Evolutionary Algorithm
def generate_initial_population(population_size, num_uavs, grid_size):
    population = []
    for _ in range(population_size):
        waypoints = [(random.randint(0, grid_size-1), random.randint(0, grid_size-1)) for _ in range(num_uavs)]
        route = UAVRoute(waypoints)
        population.append(route)
    return population

def evaluate_fitness(route, environment):
    # Define a fitness function based on objectives and constraints
    # For simplicity, let's consider the total distance traveled as the fitness
    total_distance = sum(abs(route.waypoints[i][0] - route.waypoints[i+1][0]) +
                         abs(route.waypoints[i][1] - route.waypoints[i+1][1])
                         for i in range(len(route.waypoints) - 1))
    route.fitness = -total_distance  # Negative because we want to minimize distance

# Dyna-Q Algorithm
class DynaQAgent:
    def __init__(self, num_actions, learning_rate, discount_factor):
        self.q_values = {}  # State-action values
        self.model = {}  # Model of the environment
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.num_actions = num_actions

    def update_q_values(self, state, action, reward, next_state):
        # Q-learning update rule
        current_q = self.q_values.get((state, action), 0)
        max_future_q = max(self.q_values.get((next_state, a), 0) for a in range(self.num_actions))
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_future_q - current_q)
        self.q_values[(state, action)] = new_q

        # Model update
        self.model[(state, action)] = (reward, next_state)

    def plan(self, state, num_iterations):
        # Dyna-Q planning using the learned model
        for _ in range(num_iterations):
            sampled_state, sampled_action = random.choice(list(self.model.keys()))
            reward, next_state = self.model[(sampled_state, sampled_action)]
            self.update_q_values(sampled_state, sampled_action, reward, next_state)

# Main Hybrid Algorithm
def hybrid_algorithm(population_size, num_uavs, grid_size, num_generations, dyna_q_iterations):
    env = Environment(grid_size)
    env.add_obstacle((1, 1))  # Add an obstacle at position (1, 1) for illustration

    population = generate_initial_population(population_size, num_uavs, grid_size)

    dyna_q_agent = DynaQAgent(num_actions=4, learning_rate=0.1, discount_factor=0.9)

    for generation in range(num_generations):
        for route in population:
            # Evaluate fitness of the route in the current environment
            evaluate_fitness(route, env)

            # Update Dyna-Q agent based on the current route
            for i in range(len(route.waypoints) - 1):
                state = route.waypoints[i]
                action = i  # For simplicity, use the waypoint index as the action
                reward = route.fitness  # Use fitness as the reward
                next_state = route.waypoints[i+1]
                dyna_q_agent.update_q_values(state, action, reward, next_state)

        # Select the best routes from the population
        population = sorted(population, key=lambda x: x.fitness, reverse=True)[:population_size]

        # Update routes using Dyna-Q planning
        for route in population:
            for _ in range(dyna_q_iterations):
                for i in range(len(route.waypoints) - 1):
                    state = route.waypoints[i]
                    action = i
                    dyna_q_agent.plan(state, num_iterations=1)
                    # Adjust route based on Q-values or other criteria if needed

    # Return the best route from the final population
    best_route = max(population, key=lambda x: x.fitness)
    return best_route

# Example usage
best_route = hybrid_algorithm(population_size=10, num_uavs=50, grid_size=5, num_generations=15, dyna_q_iterations=10)
print("Best Route:", best_route.waypoints)
print("Fitness:", best_route.fitness)


Best Route: [(4, 1), (3, 3), (4, 1), (2, 1), (2, 2), (1, 0), (4, 3), (2, 3), (3, 3), (3, 2), (4, 2), (1, 1), (1, 3), (2, 2), (0, 2), (4, 1), (0, 3), (4, 2), (1, 0), (2, 3), (2, 2), (2, 0), (4, 3), (2, 2), (0, 1), (2, 0), (0, 2), (1, 1), (4, 2), (4, 4), (2, 0), (0, 0), (2, 2), (0, 2), (3, 1), (4, 4), (3, 0), (1, 0), (4, 3), (0, 3), (2, 3), (3, 3), (3, 4), (1, 3), (1, 4), (1, 2), (4, 1), (3, 1), (0, 3), (4, 4)]
Fitness: -151
